In [3]:
import matplotlib
#matplotlib.use('Agg')
%matplotlib tk
%autosave 180
import matplotlib.pyplot as plt
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

import matplotlib.cm as cm
from matplotlib import gridspec
from scipy import signal

import numpy as np
import os
import shutil
import cv2
from tqdm import trange

#import glob2

from scipy.io import loadmat
import scipy
#import hdf5storage
import csv


Autosaving every 180 seconds


In [4]:
# Classifier output for CSUA; using 100 x 100 pixel images;
fname_classification = '/media/cat/4TBSSD/dan/march_2/madeline_dlc/march_16/2020-3-16_12_54_07_193951_compressed/training_images/classification_output.npz'
                        #/media/cat/4TBSSD/dan/march_2/madeline_dlc/march_16/2020-3-16_12_54_07_193951_compressed/training_images/classification_output.npz

#            
locs = np.load(os.path.split(fname_classification)[0]+'/locs_array_clean.npy')
print (locs.shape)

FileNotFoundError: [Errno 2] No such file or directory: '/media/cat/4TBSSD/dan/march_2/madeline_dlc/march_16/2020-3-16_12_54_07_193951_compressed/training_images/locs_array_clean.npy'

In [107]:
############################################
############ VISUALIZE TRACES ##############
############################################
clrs = ['blue','red','yellow','green','cyan']
names= ['female','male','pup1','pup2','block']
window = 10 * 25
metre_per_pixel = 0.5*1E-3

for p in range(4):
    
    # compute differences as movement
    positions = locs[p].sum(1)
    t = np.arange(positions.shape[0])/25.
    print (p, positions)
    plt.plot(t, positions, linewidth=3, c=clrs[p], label=names[p])
plt.legend(fontsize=20)
plt.show()

0 [990.281   985.28894 984.965   ... 989.746   990.62195 990.281  ]
1 [ 910.481   1329.049   1329.2051  ...  910.92505  910.315    910.481  ]
2 [928.552   928.552   927.95996 ... 832.254   832.254   832.254  ]
3 [942.734   942.734   941.673   ... 992.45996 992.45996 992.45996]


In [114]:
############################################
############ MAKE VIDEO BASES ON TRACES ####
############################################
#          pup1     pup2    female  male
# go through first videos
from tqdm import trange

colors_4= ['orange','green', 'blue', 'red', 'cyan']
colors_4= ['blue','red', 'cyan', 'green', 'yellow']

video_name = '/media/cat/4TBSSD/dan/march_2/madeline_dlc/march_16/2020-3-16_12_54_07_193951_compressed/2020-3-16_12_54_07_193951_compressed.avi'
original_vid = cv2.VideoCapture(video_name)
font = cv2.FONT_HERSHEY_PLAIN

# video sizes
size_vid = np.array([1280,1024])
scale = 1
dot_size = 8//scale

#out_dir = '/media/cat/4TBSSD/dan/march_2/madeline_dlc/2020-3-9_08_18_49_128168/'

start = 18000
end = start+4000
original_vid.set(cv2.CAP_PROP_POS_FRAMES, start)

# Initialize video out
fname_out = video_name[:-4]+"_locs_added_"+str(start)+"_"+str(end)+".mp4"
fourcc = cv2.VideoWriter_fourcc('M','P','E','G')
video_out = cv2.VideoWriter(fname_out,fourcc, 25, (size_vid[0]//scale,size_vid[1]//scale), True)


for n in trange(start,end, 1):
    ret, frame = original_vid.read()
    cv2.putText(frame, str(n), (50, 100), font, 5, (255, 255, 0), 5)
    frame = frame[::scale, ::scale]
    
    for k in range(5):
        y = locs[k,n,0]
        x = locs[k,n,1]
        
        if np.isnan(x) or np.isnan(y):
            continue
        else:
            x=int(x)//scale
            y=int(y)//scale
            
            frame[x-dot_size:x+dot_size,y-dot_size:y+dot_size]= (np.float32(
                matplotlib.colors.to_rgb(colors_4[k]))*255.).astype('uint8')
            #print (colors_4[k])
            #frame[y-dot_size:y+dot_size,x-dot_size:x+dot_size]= (np.float32(
            #    matplotlib.colors.to_rgb(colors_4[z//14]))*255.).astype('uint8')
                
    #print ("")
    video_out.write(frame)

    #print ("")

video_out.release()
original_vid.release()
#cv2.destroyAllWindows()



100%|██████████| 2000/2000 [00:25<00:00, 79.07it/s]


In [32]:
############################################
####### MAKE BEHAVIOUR CYCLE PLOTS #########
############################################

clrs = ['blue','red','yellow','green','yellow']
names= ['female','male','pup1','pup2','block']
window = 60 * 25
metre_per_pixel = 0.5*1E-3

for p in range(4):
    
    # compute differences as movement
    positions = locs[p]
    velocities = np.sqrt((positions[1:,0]-positions[:-1,0])**2+
                         (positions[1:,1]-positions[:-1,1])**2)
    print (velocities)
                         
    ynew = np.convolve(velocities, np.ones((window,))/window, mode='valid')
    t = np.arange(ynew.shape[0])/25.
        
    dists = np.sum(ynew)
    print (dists)
    
    plt.plot(t, ynew, linewidth=3, c=clrs[p], label=names[p]+" dist: "+str(round(dists*metre_per_pixel,1)) + " m")

# label information
plt.ylabel("Activity", fontsize=30)
plt.yticks([0],fontsize=30)
plt.legend(fontsize=20)
plt.xlabel("Time (sec)",fontsize=20)
plt.xlim(t[0],t[-1])
plt.ylim(bottom=0)
plt.suptitle("Movement analysis for Cohort #1 - March 16: 2020-3-16_12_54_07_193951_compressed\n"+
             str(int(window/25))+"sec smoothing window",fontsize=25)
plt.show()
    

[49.307186    0.256755    0.39745963 ...  0.317553    0.63351184
  0.2880867 ]
68411.0544567663
[1.0581298e+03 8.5713798e-01 6.0084014e+00 ... 4.9054497e-01 1.0746862e+00
 1.6501181e-01]
184446.57650995554
[0.         0.42713147 0.14088905 ... 0.         0.         0.        ]
146750.93679426177
[0.        1.2314171 0.7045507 ... 0.        0.        0.       ]
97276.96287562922


In [94]:
###############################################
# COMPUTE SOCIAL INTERACTIONS BETWEEN ANIMALS###
################################################
from mpl_toolkits.axes_grid1 import make_axes_locatable
from itertools import combinations

x_ticks=['female','male','pup1','pup2', 'sum']
text_clr = 'red'

distance_threshold = 250 # # of pixels away assume 1 pixel ~= 0.5mm -> 20cm
time_window = 1*25 # no of seconds to consider
smoothing_window = 3
min_distance = 25 # number of frames window

traces_23hrs = locs

print (traces_23hrs[0].shape)
# plt.plot(traces_2D[0][0])
# plt.plot(traces_2D[0][1])

#############################
# COMPUTE PAIRWISE INTERACTIONS
animals=np.arange(4)
interactions = np.zeros((animals.shape[0],animals.shape[0]),'int32') + np.nan
durations_matrix = np.zeros((animals.shape[0], animals.shape[0]),'int32') + np.nan

pairs1 = list(combinations(animals,2))
for ctr_x, pair in enumerate(pairs1):
    traces = []
    # COMPUTE LOCATIONS
    for k in pair:
        traces1=traces_23hrs[k].copy()
        traces1[:,0]=np.convolve(traces_23hrs[k,:,0], np.ones((smoothing_window,))/smoothing_window, mode='same')
        traces1[:,1]=np.convolve(traces_23hrs[k,:,1], np.ones((smoothing_window,))/smoothing_window, mode='same')
        traces1 = traces1
        traces.append(traces1)
    
    # COMPUTE PAIRWISE DISTANCES AND NEARBY TIMES POINTS
    idx_array = []
    diffs = np.sqrt((traces[0][:,0]-traces[1][:,0])**2+
                    (traces[0][:,1]-traces[1][:,1])**2)
    idx = np.where(diffs<distance_threshold)[0]

    # COMPUTE TOTAL TIME TOGETHER
    #print ("Pairwise: ", pair, idx.shape)
    durations_matrix[pair[0],pair[1]]=idx.shape[0]/25

    # COMPUTE # OF INTERACTIONS
    diffs_idx = idx[1:]-idx[:-1]
    idx2 = np.where(diffs_idx>5)[0]
    interactions[pair[0],pair[1]]=idx2.shape[0]

# SYMMETRIZE MATRICES
for k in range(durations_matrix.shape[0]):
    for j in range(durations_matrix.shape[1]):
        if np.isnan(durations_matrix[k,j])==False:
            durations_matrix[j,k]=durations_matrix[k,j]
            interactions[j,k]=interactions[k,j]


#############################
# COMPUTE TRIPLE INTERACTIONS

multi_animal_durations = [] #np.zeros((animals.shape[0]+4),'int32') + np.nan
names_multi_animal =[]
ctr=0

pairs3 = list(combinations(animals,3))
for ctr_x, pair in enumerate(pairs3):
    names_multi_animal.append(names[pair[0]]+"\n"+ names[pair[1]]+"\n"+names[pair[2]])
    traces = []
    
    # COMPUTE LOCATIONS
    for k in pair:
        traces1=traces_23hrs[k].copy()
        traces1[:,0]=np.convolve(traces_23hrs[k,:,0], np.ones((smoothing_window,))/smoothing_window, mode='same')
        traces1[:,1]=np.convolve(traces_23hrs[k,:,1], np.ones((smoothing_window,))/smoothing_window, mode='same')
        traces1 = traces1
        traces.append(traces1)
    
    # COMPUTE PAIRWISE DISTANCES AND NEARBY TIMES POINTS
    idx_array = []
    pairs2 = list(combinations(np.arange(3),2))
    for pair_ in pairs2:
        #print ("pair_", pair_)
        diffs = np.sqrt((traces[pair_[0]][:,0]-traces[pair_[1]][:,0])**2+
                        (traces[pair_[0]][:,1]-traces[pair_[1]][:,1])**2)
        idx_temp = np.where(diffs<distance_threshold)[0]
        print ("pair_: ", pair_, idx_temp.shape)
        idx_array.append(idx_temp)

    # COMPUTE OVERLAP 
    idx3 = np.unique(np.hstack(set.intersection(*[set(x) for x in idx_array])))
    print (pair, "IDX3 4: ", len(idx3))
    multi_animal_durations.append(len(idx3)/25)
 
print ("")
 #################################
# COMPUTE 4 ANIMAL INTERACTIONS
pairs = list(combinations(animals,4))
for ctr_x, pair in enumerate(pairs):
    names_multi_animal.append(names[pair[0]]+"\n"+ names[pair[1]]+"\n"+names[pair[2]]+"\n"+names[pair[3]])
    print (pair)
    traces = []
    for k in pair:
        traces1=traces_23hrs[k].copy()
        traces1[:,0]=np.convolve(traces_23hrs[k,:,0], np.ones((smoothing_window,))/smoothing_window, mode='same')
        traces1[:,1]=np.convolve(traces_23hrs[k,:,1], np.ones((smoothing_window,))/smoothing_window, mode='same')
        traces1 = traces1
        traces.append(traces1)
    
    # loop over all combinations and get distances
    pairs2 = list(combinations(animals,2))
    idx_array = []
    for pair2 in pairs2:
        diffs = np.sqrt((traces[pair2[0]][:,0]-traces[pair2[1]][:,0])**2+
                         (traces[pair2[0]][:,1]-traces[pair2[1]][:,1])**2)
        idx_temp = np.where(diffs<distance_threshold)[0]
        print ("pair2: ", pair2, idx_temp.shape)
        idx_array.append(idx_temp)

    # COMPUTE TOTAL TIME TOGETHER
    print ("idx_array: ", idx_array)
    idx3 = set.intersection(*[set(x) for x in idx_array])
    print ("IDX3 4: ", len(idx3))
    multi_animal_durations.append(len(idx3)/25)

    ctr+=1
print ("")
 
###############################################################
# PLOT MULTI ANIMAL DURATIONS
ax3=plt.subplot(2,2,3)
plt.bar(np.arange(len(multi_animal_durations)),
         multi_animal_durations, 0.9)

plt.xticks(np.arange(len(multi_animal_durations)),
          names_multi_animal)

ax3.set_title("Duration Multi-animal interactions",fontsize=15)
#################################################
######### PLOT INTERACTIONS PAIRWISE ############
#################################################

ax1=plt.subplot(2,2,1)

im = plt.imshow(interactions)
plt.xticks(np.arange(4), x_ticks)
plt.yticks(np.arange(4), x_ticks)

ax1.set_title("# of interactions between animals",fontsize=20)

# Loop over data dimensions and create text annotations.
for i in range(interactions.shape[0]):
    for j in range(interactions.shape[1]):
        if np.isnan(durations_matrix[i, j])==True:
            continue
        text = ax1.text(j, i, interactions[i, j],
                       ha="center", va="center", color=text_clr,
                       fontsize=13)        

##############################################
############ PLOT PAIRWISE DURATIONS ########
#################################################
ax2=plt.subplot(2,2,2)
im = plt.imshow(durations_matrix, cmap='viridis')


x_ticks=['female','male','pup1','pup2']
plt.xticks(np.arange(4), x_ticks)
plt.yticks(np.arange(4), x_ticks)

# Loop over data dimensions and create text annotations.
for i in range(durations_matrix.shape[0]):
    for j in range(durations_matrix.shape[1]):
        if np.isnan(durations_matrix[i, j])==True:
            continue
        text = ax2.text(j, i, durations_matrix[i, j],
                       ha="center", va="center", color=text_clr,
                       fontsize=13)

ax2.set_title('Total time spent together (seconds)',fontsize=20)

plt.suptitle("Behav analysis for Cohort #1\n March 16: 2020-3-16_12_54_07_193951_compressed (1hr video; "+
             str(np.round(smoothing_window/25,2))+"sec smoothing window)",fontsize=15)

plt.show()   
    
#cax = fig.add_axes([0.27, 0.8, 0.5, 0.05])
    

(89988, 2)
pair_:  (0, 1) (64837,)
pair_:  (0, 2) (54133,)
pair_:  (1, 2) (49821,)
(0, 1, 2) IDX3 4:  44651
pair_:  (0, 1) (64837,)
pair_:  (0, 2) (9183,)
pair_:  (1, 2) (7844,)
(0, 1, 3) IDX3 4:  1414
pair_:  (0, 1) (54133,)
pair_:  (0, 2) (9183,)
pair_:  (1, 2) (20565,)
(0, 2, 3) IDX3 4:  2851
pair_:  (0, 1) (49821,)
pair_:  (0, 2) (7844,)
pair_:  (1, 2) (20565,)
(1, 2, 3) IDX3 4:  1222

(0, 1, 2, 3)
pair2:  (0, 1) (64837,)
pair2:  (0, 2) (54133,)
pair2:  (0, 3) (9183,)
pair2:  (1, 2) (49821,)
pair2:  (1, 3) (7844,)
pair2:  (2, 3) (20565,)
idx_array:  [array([ 3365,  3366,  3367, ..., 89985, 89986, 89987]), array([  125,   126,   127, ..., 89985, 89986, 89987]), array([    0,     1,     2, ..., 88986, 88987, 88988]), array([    0,     1,   205, ..., 89985, 89986, 89987]), array([    0,   797,   798, ..., 89369, 89370, 89371]), array([    0,    46,    47, ..., 89757, 89758, 89759])]
IDX3 4:  478



/home/cat/.conda/envs/DLC-GPU/lib/python3.7/site-packages/ipykernel_launcher.py:93: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.


In [86]:
######################################################
####### MOVEMENT INTERACTION INITIATIONS #############
######################################################





In [112]:
######################################################
######### SOCIAL INTERACTION INITIATIONS #############
######################################################



In [27]:
# BEHAVIUR ANALYSIS - POST TRACE ID SWAP FIXING

data = np.load('/media/cat/4TBSSD/dan/march_2/madeline_dlc/march_16/2020-3-16_12_54_07_193951_compressed/2020-3-16_12_54_07_193951_compressedvideo_labeled_exported_corrected.npz',
               allow_pickle=True)
tracesx = data['tracesx']
tracesy = data['tracesy']
print (tracesx)


# plot time series
clrs = ['red','blue','cyan','green']
# from scipy.ndimage import gaussian_filter1d
# from scipy import interpolate

window = 25 * 300 # seconds
#ax=plt.subplot(1,1,1)

ctr=0

# initialize arrays to hold data
# sums_array = []
# sums_individuals = []
# for k in range(4):
#     sums_individuals.append([])
# print ("sums individualsl" , sums_individuals)
# print ("Loading: ", fname, tracesx.shape)

traces = np.zeros((tracesx.shape[0],tracesx.shape[1],2),'float32')
traces[:,:,0]=tracesx
traces[:,:,1]=tracesy
print ("TRACES: ", traces.shape)

idx = np.where(traces==0)
traces[idx]=np.nan

# Load trace info
sums=np.zeros(traces.shape[0]+10,'float32')
for p in range(4):
    
    # find nans and interpolate over them
    # locs = np.nanmean(np.nanmean(np.abs(traces[p*14: (p+1)*14,:,:]),2),0)
    locs = traces[:,p,0]
    x = np.where(np.isnan(locs)==False)[0]
    
    # fill in missing data with linear interpolation
    xnew = np.arange(x[0],x[-1]+1,1)
    fx = interpolate.interp1d(x, traces[x,p,0])
    fy = interpolate.interp1d(x, traces[x,p,1])
    
    ynew1 = fx(xnew)   # use interpolation function returned by `interp1d`
    ynew2 = fy(xnew)   # use interpolation function returned by `interp1d`
    
    dists = np.sqrt((ynew1[1:]-ynew1[:-1])**2+(ynew2[1:]-ynew2[:-1])**2)
    # plt.plot(xnew/25.+ctr*3600, ynew)
    #plt.plot(xnew, ynew)
    # compute differences as movement
    #ynew1 = np.abs(ynew[1:]-ynew[:-1])
    ynew = dists
    
    # instead, just grab all the data and reprocess it below
    #sums[xnew[:ynew.shape[0]]]+=ynew*10
    #print ("sums: ", sums.shape, xnew.shape, ynew.shape)
    sums[xnew[:-1]]+=ynew

    sums_individuals[p] = ynew
    print ("len:", len(sums_individuals[p]))

print ("SUMS: ", sums.shape)
# ynew = np.hstack(sums_array)
ynew = sums

ynew = np.convolve(ynew, np.ones((window,))/window, mode='valid')
print ("ynew: ", ynew.shape)
t = np.arange(ynew.shape[0])/25.+1*3600
# plt.plot(t, ynew, linewidth=3, c='black', label="Sum")

window = 25*1
final_traces = []
for p in range(4):
    print ("plotting animal:", p)
    ynew = sums_individuals[p]
    print ("Ynew: ", ynew)
    ynew = np.convolve(ynew, np.ones((window,))/window, mode='valid')
    t = np.arange(ynew.shape[0])/25.#+1*3600
    final_traces.append(ynew)

print ("Final traces: ", len(final_traces))
    

# label information
plt.ion()
fig, ax = plt.subplots()
r = plt.gcf().canvas.get_renderer()
#x = im.make_image(r, magnification=2.0)


plt.ylabel("Activity", fontsize=30)
plt.yticks([0,10,20,30,40],fontsize=30)
#xticks_old = np.arange(0*3600,3600*24, 3600)
#xticks_new = np.arange(24)
#plt.xticks(xticks_old,xticks_new,fontsize=30)
#plt.xlabel("Time of Day (24hrs clock)",fontsize=30)
# ax.axvspan(0, 7*3600, alpha=0.1, color='black')
# ax.axvspan(19*3600,24*3600, alpha=0.1, color='black')
#plt.xlim(0,60)
#plt.ylim(bottom=0)
plt.xticks(fontsize=30)
plt.xlabel("Time (sec)",fontsize=30)
plt.ylim(0,50)
plt.suptitle("Movement analysis for Cohort #1 - March 16\n"+
             str(window/25)+"sec smoothing window",fontsize=25)

k=0
lines = []
for p in range(4):
    temp, = ax.plot(t[k:k+25*60], final_traces[p][k:k+25*60], linewidth=3, c=clrs[p], label=names[p])
    lines.append(temp)
plt.legend(fontsize=20,loc=1)
    
for k in range(1,3600*25,25):
    for p in range(4):
        lines[p].set_ydata(final_traces[p][k:k+25*60])
        lines[p].set_xdata(np.arange(k,k+25*60,1)/25)
        
    fig.canvas.draw()
    fig.canvas.flush_events()
    plt.xlim(k/25,(k+25*60)/25)
    #.setdata(ynew[k:k+25*60])
    print (k)

plt.show()
    
    



[[       nan 1142.6586         nan  421.74698]
 [       nan 1135.3518         nan  419.19302]
 [       nan 1139.0096         nan  416.82108]
 ...
 [ 300.28156        nan  263.7146         nan]
 [ 301.60367        nan  244.42133  325.44324]
 [ 307.49664        nan  243.933    323.9843 ]]
TRACES:  (89988, 4, 2)


NameError: name 'interpolate' is not defined

In [96]:
for p in range(4):
    temp = np.sum(final_traces[p])
    temp = temp/25 / 100  #convert to meters
    plt.bar(p, temp,0.9, color=clrs[p], label=names[p])
    plt.ylabel("# of Metres traveled in 1 hour",fontsize=30)
plt.legend(fontsize=30,loc=1)
plt.xticks([])
plt.yticks(fontsize=30)
plt.show()

In [2]:
data = np.load('/media/cat/4TBSSD/dan/march_2/madeline_dlc/march_16/2020-3-16_12_54_07_193951_compressed/training_images_cc/segments_all/classification_output.npz',
              allow_pickle=True)
vals = data['vals']
frames = np.hstack(data['frames_array'])
print ("# of lableed frames: ", vals.shape)
classes = data['classes']
names = ['female','male','pup1','pup2','block']
clrs = ['red','blue','cyan','green','yellow']

print (" # of labeled segments: ", np.unique(vals).shape)
n_animals = 5
single_ids = 0
threshold = 0.7
correct_ids = 0
plotting = False
if plotting:
    fig=plt.figure()
percent_array = []
lengths = []
for k in range(5):
    lengths.append([])
# 
ctr=0
vals_in = np.unique(vals)
#vals_in = np.sort(np.random.choice(vals_in, 400, replace=False))
for dir_ in vals_in:
    idx = np.where(vals==dir_)[0]
    
    print (idx.shape)
    temp_classes = classes[idx]
    # 
    
        
    if plotting:
        if ctr>29:
            break

        if idx.shape[0]<30:
            continue

        ax=plt.subplot(5,6,ctr+1)
    #y = np.histogram(temp_classes, np.arange(-0.5,3.5))
    
    temp_ctr=0
    id_ = None
    for k in np.unique(temp_classes):
        idx=np.where(temp_classes==k)[0]
        #print (names[k], idx.shape)
        if plotting:
            plt.bar(k,idx.shape[0],.9, color=clrs[k])
    
        if idx.shape[0]>temp_ctr:
            temp_ctr = idx.shape[0]
            id_ = k
            
    lengths[id_].append(temp_classes.shape[0])

    # compute % for best match
    percent = round((np.where(temp_classes==id_)[0].shape[0])/temp_classes.shape[0],2)
    percent_array.append(percent)
    
    #
    if plotting:
        plt.title("seg: "+str(int(dir_))+", "+names[id_]+ " "+str(round(percent*100,2))+"%", fontsize=8)
        plt.xticks([])
        plt.xlim(-0.5,4.5)

            
    if percent>threshold:
        single_ids+=1
        correct_ids+=np.where(temp_classes==id_)[0].shape[0]
        
    ctr+=1
    #if ctr==1:
    #    break
    
print ("% correct (segments): ", single_ids/ np.unique(vals).shape[0])
print ("% correct (individual frame): ", correct_ids/ vals.shape[0])
if plotting:
    plt.show()
    
    


# of lableed frames:  (39225,)
 # of labeled segments:  (1264,)
(229,)
(351,)
(4346,)
(374,)
(9,)
(9,)
(10,)
(4,)
(2,)
(7,)
(5,)
(2,)
(2,)
(4,)
(2,)
(4,)
(7,)
(2,)
(8,)
(1,)
(3,)
(2,)
(20,)
(2,)
(6,)
(2,)
(3,)
(2,)
(3,)
(5,)
(2,)
(3,)
(1,)
(30,)
(4,)
(2,)
(25,)
(5,)
(2,)
(11,)
(9,)
(24,)
(1,)
(4,)
(223,)
(19,)
(1,)
(2,)
(43,)
(1,)
(2,)
(469,)
(2,)
(1,)
(1,)
(2,)
(2,)
(126,)
(3,)
(1,)
(4,)
(1,)
(9,)
(1,)
(3,)
(4,)
(6,)
(2,)
(1,)
(53,)
(5,)
(1,)
(2,)
(1,)
(3,)
(1,)
(1,)
(6,)
(151,)
(50,)
(33,)
(2,)
(5,)
(21,)
(2087,)
(1,)
(122,)
(1,)
(2,)
(4,)
(4,)
(1,)
(1,)
(2,)
(3,)
(1,)
(1,)
(2,)
(6,)
(1,)
(52,)
(3,)
(2,)
(1,)
(1,)
(2,)
(4,)
(26,)
(1,)
(3,)
(8,)
(3,)
(2,)
(4,)
(5,)
(73,)
(4,)
(12,)
(4,)
(15,)
(1,)
(1,)
(1,)
(2,)
(3,)
(4,)
(1,)
(2,)
(16,)
(1,)
(1,)
(132,)
(2,)
(1,)
(5,)
(11,)
(1,)
(9,)
(3,)
(4,)
(1,)
(1,)
(3,)
(4,)
(1,)
(1,)
(4,)
(3,)
(2,)
(14,)
(1,)
(2,)
(1,)
(1,)
(16,)
(9,)
(3,)
(4,)
(12,)
(2,)
(3,)
(9,)
(35,)
(101,)
(2,)
(1,)
(4,)
(1,)
(59,)
(1,)
(66,)
(59,)
(92,)
(2,)
(11,)
(2,)
(6

In [3]:
#data = np.load('/media/cat/4TBSSD/dan/march_2/madeline_dlc/march_16/2020-3-16_12_54_07_193951_compressed/training_images_cc/segments_all/frames_array.npy', 
             # allow_pickle=True)
times = frames
print (np.unique(times).shape)
#print (np.unique(classes).shape)
#print (frames)
#percentages_final = []
for k in range(5):
    idx = np.where(classes==k)[0]
    plt.scatter(times[idx]/25., classes[idx], c=clrs[k])
    print (names[k], idx.shape[0], np.unique(times[idx]).shape)
    
    
    
plt.show()

(9999,)
female 10736 (9522,)
male 11964 (9162,)
pup1 6588 (5776,)
pup2 4121 (3737,)
block 5816 (5740,)


In [21]:
# fig = plt.figure()
# print (len(percent_array))
# plt.title("Distribution of the percent correct per segment")
# y = np.histogram(percent_array, np.arange(0,1.1, 0.1))
# plt.bar(y[1][:-1],y[0], 0.09)

fig = plt.figure()
width=9
width2 =  0.19
frame_rate=25
plt.title("Distribution of lengths of segment (total rec: "+str(round(10000/frame_rate,2))+" sec)")
for k in range(4):
    temp = np.array(lengths[k])/25
    y = np.histogram(temp, np.arange(0,400,1))

    idx = np.where(classes==k)[0]
    #plt.scatter(times[idx]/25., classes[idx], c=clrs[k])
    total_dur = np.unique(times[idx]).shape[0]/frame_rate
    
    
    if False:
        if k<1:
            plt.bar(y[1][:-1], y[0], width, color=clrs[k])
            dataset_old = y[0].copy()
        else:
            plt.bar(y[1][:-1], y[0], width, bottom=dataset_old, color=clrs[k])

            dataset_old = y[0].copy()+dataset_old
    else:
        #plt.bar(y[1][:-1]+width2*k+0.5, y[0], width2, color=clrs[k])
        plt.plot(y[1][:-1]+width2*k+0.5, y[0], color=clrs[k], 
                 label=names[k]+" "+str(total_dur)+"sec")

        
        
    #p2 = plt.bar(ind, dataset2, width, bottom=dataset1, color='b')
plt.legend(fontsize=20)
plt.semilogy(bottom=1E-2)
plt.semilogx(bottom=1E-2)
plt.xlabel("Length of time segment (seconds)")

plt.show()

In [20]:
animal_arrays = np.load('/media/cat/4TBSSD/dan/march_2/madeline_dlc/march_16/2020-3-16_12_54_07_193951_compressed/animal_arrays.npy')
print (animal_arrays.shape)

# plt.plot(animal_arrays[3,:].sum(1))
# plt.show()

(5, 89988, 2)


In [43]:
#################################################################
###### CLEANUP: Del short segs; connect safe segs ###############
#################################################################

animal_arrays = np.load('/media/cat/4TBSSD/dan/march_2/madeline_dlc/march_16/2020-3-16_12_54_07_193951_compressed/animal_arrays.npy')

clrs = ['red','blue','cyan','green','yellow']
clrs_edit = ['black','blue','red','green','magenta']

# REMOVE GARBAGE / FAILED PREDICTION
idx = np.where(animal_arrays==0.)
animal_arrays[idx]=np.nan


# DEL SHORT SEGS
min_len_seg = 5
ks = [2]
#ks = np.arange(5)
for k in ks:
    plt.plot(animal_arrays[k,:].sum(1), c=clrs_edit[0])

    temp = animal_arrays[k]
    print (temp.shape)
    
    idx = np.where(np.isnan(temp[:,0])==True)[0]
    print (idx.shape)
    
    diffs = idx[1:]-idx[:-1]
    idx2 = np.where(np.logical_and(diffs>1, diffs<min_len_seg))[0]
    print ("short segs: ", diffs[idx2], idx2.shape)
    
    # loop over short segs and remove them
    for id_ in idx2:
        print (idx[id_-1],idx[id_])
        temp[idx[id_]:idx[id_+1]]=np.nan
    print (idx2.shape)

    #idx = np.where(temp[:,0]==0)[0]
    #temp[idx]=np.nan
    
    plt.plot(np.sum(temp, axis=1)+2, c=clrs_edit[1])


# VELOCITIES

#     temp = temp[1:]-temp[:-1]
    
#     temp = np.sqrt(temp[:,0]**2+temp[:,1]**2)
#     print (temp.shape)
#     #idx = np.where(temp==0)[0]
#     #temp[idx]=np.nan
#     idx = np.where(temp>300)[0]
#     temp[idx]=np.nan

    
    
plt.show()


(89988, 2)
(32870,)
short segs:  [4 2 2 ... 2 2 2] (1144,)
958 959
966 967
1055 1056
1284 1377
1380 1381
1381 1384
1384 1387
4281 4282
4356 4357
4825 4826
4933 4934
5034 5035
5038 5039
5061 5062
5109 5110
8506 8507
8535 8536
8549 8550
8574 8575
8602 8603
8771 8772
8788 8789
8792 8793
8817 8818
8836 8837
8884 8885
8887 8888
8953 8954
8958 8959
8973 8974
9018 9019
9048 9049
12907 12908
17948 18149
18153 18154
19982 19983
19994 19995
19998 19999
22505 22506
22753 22754
22766 22767
22774 22775
22883 22894
22894 22896
22898 22899
22953 22954
22991 22992
23197 23198
24147 24148
24157 24158
24179 24180
24180 24183
24186 24187
24187 24191
24191 24193
24242 24243
24257 24258
24262 24263
24267 24268
24272 24273
24282 24283
24302 24303
24307 24308
24312 24313
24346 24347
24362 24363
24417 24418
24422 24423
24430 24431
24431 24433
24442 24443
24446 24447
24453 24458
24458 24460
24462 24463
24473 24474
24478 24479
24500 24506
25714 26510
26514 26515
27144 27145
28688 28780
29652 29653
29656 29657
2